## PopCount8 and PopCount

In this tutorial, we will illustrate how `Python` can be used to 
construct `Magma Circuits`.

We use Wallace Trees to construct a `PopCount` circuit, 
which counts the number of bits that are set in an n-bit value.

In [1]:
import magma as m
m.set_mantle_target("ice40")

In this example, we are going to use the built-in `fulladder` from `Mantle`.
`fulladder` instantiates a 3-input 2-output and wires up the inputs and the outputs.
A common name for a full adder is a carry-sum adder, `csa`.

In [2]:
from mantle import fulladder
csa = fulladder

import lattice ice40
import lattice mantle40


To construct the 8-bit popcount, we first use 3 fulladders to sum
bits 0 through 2, 3 through 5, and 6 through 7.
This forms 3 2-bit results.
We can consider the results to be two columns, one for each *place*.
The first column is the 1s and the second column is the 2s.
We then use two fulladders to sum these columns.
We continue summing 3-bits at a time until we get a single bit in each column.

A common way to show these operations is with *Dadda dot notation*
which shows how many bits are in each colum.

In [3]:
def popcount8(I):
    # Dadda dot notation (of the result)
    # o o
    # o o
    # o o
    csa0_0_21 = csa(I[0], I[1], I[2])
    csa0_1_21 = csa(I[3], I[4], I[5])
    csa0_2_21 = csa(I[6], I[7], 0)

    #   o o
    # o o
    csa1_0_21 = csa(csa0_0_21[0], csa0_1_21[0], csa0_2_21[0])
    csa1_0_42 = csa(csa0_0_21[1], csa0_1_21[1], csa0_2_21[1])

    #     o
    # o o o
    csa2_0_42 = csa(csa1_0_21[1], csa1_0_42[0], 0)

    # o o o o
    csa2_0_84 = csa(csa1_0_42[1], csa2_0_42[0], 0)
    
    return m.bits([csa1_0_21[0], csa2_0_42[0], csa2_0_84[0], csa2_0_84[1]])

## Test bench

In order to test the popcount circuit,
we setup the IceStick board
to have eight inputs and four outputs.
As before, `J1` will be used for inputs and `J3` for outputs.

In [4]:
from loam.boards.icestick import IceStick

icestick = IceStick()
for i in range(8):
    icestick.J1[i].input().on()
for i in range(4):
    icestick.J3[i].output().on()
    
main = icestick.DefineMain()

m.wire( popcount8(main.J1), main.J3 )

m.EndDefine()

In [5]:
m.compile('build/popcount8', main)

compiling FullAdder
compiling main


And use our `yosys`, `arcachne-pnr`, and `icestorm` tool flow.

In [6]:
%%bash
cd build
yosys -q -p 'synth_ice40 -top main -blif popcount8.blif' popcount8.v
arachne-pnr -q -d 1k -o popcount8.txt -p popcount8.pcf popcount8.blif 
icepack popcount8.txt popcount8.bin
iceprog popcount8.bin

/Users/hanrahan/git/magmathon/notebooks/tutorial/build


init..
cdone: high
reset..
cdone: low
flash ID: 0x20 0xBA 0x16 0x10 0x00 0x00 0x23 0x12 0x67 0x21 0x20 0x00 0x61 0x00 0x53 0x04 0x11 0x11 0x03 0xB1
file size: 32220
erase 64kB sector at 0x000000..
programming..
reading..
VERIFY OK
cdone: high
Bye.


You can test the program by connecting up some switches and LEDs to the headers. You should see the count of the inputs displayed on the LEDs.

There is a more general version of `PopCount` in the `Mantle` library `util.compressor`.